In [1]:
import json
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
def obter_dados(fonte, fonte_amigos=None):
    if fonte == 'json':
        with open('cursos.json') as f:
            dados = json.load(f)
    elif fonte == 'webhook':
        url = 'https://exemplo.com/api/dados'
        response = requests.get(url)
        dados = response.json()
    else:
        raise ValueError('Fonte de dados inválida')

    if fonte_amigos == 'json':
        with open('amigos.json') as f:
            amigos = json.load(f)
    elif fonte_amigos == 'webhook':
        url = 'https://exemplo.com/api/amigos'
        response = requests.get(url)
        amigos = response.json()
    else:
        amigos = []

    return dados, amigos

In [3]:
fonte = input('Digite a fonte dos dados (json ou webhook): ')
fonte_amigos = input('Digite a fonte dos amigos (json, webhook ou nenhum): ')
dados, amigos = obter_dados(fonte, fonte_amigos)

In [5]:
cursos = []
amigos = []
avaliacoes = []
for usuario in dados['usuarios']:
    for curso in usuario['cursos']:
        cursos.append(curso['nome'])
        avaliacoes.append(curso['avaliacao'])
    for amigo in usuario['amigos']:
        amigos.append(amigo['nome'])
        for curso in amigo['cursos']:
            cursos.append(curso['nome'])
            avaliacoes.append(curso['avaliacao'])

In [6]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(cursos)

In [7]:
similarity_matrix = cosine_similarity(X)

In [8]:
def recomendar_cursos(usuario, pesquisas):
    idx_usuario = amigos.index(usuario)
    cursos_usuario = []
    for curso in dados['usuarios'][idx_usuario]['cursos']:
        cursos_usuario.append(curso['nome'])
    for amigo in amigos['usuarios'][idx_usuario]['amigos']:
        for curso in amigo['cursos']:
            cursos_usuario.append(curso['nome'])
    cursos_usuario += pesquisas

    curso_vec = np.zeros((1, len(vectorizer.get_feature_names())))
    for curso in cursos_usuario:
        idx_curso = vectorizer.get_feature_names().index(curso)
        curso_vec[0, idx_curso] += 1
    
    similarity_vec = cosine_similarity(curso_vec, X)

    idx_recomendados = np.argsort(similarity_vec)[0, ::-1][:10]
    
    recomendados = []
    for idx in idx_recomendados:
        recomendados.append((cursos[idx], avaliacoes[idx]))
    
    return recomendados
